In [17]:
example = "389125467"
data = "394618527"

In [23]:
from typing import Iterable, Iterator, Tuple, T

class CircularBufferNode(object):
    def __init__(self, next: "CircularBufferNode", value: T):
        self.next = next
        self.value = value

class CircularBuffer(object):
    def __init__(self, items: Iterable[T]):
        self.index = {}

        self.head = None
        tail = None
        for item in items:
            old_tail = tail
            tail = CircularBufferNode(self.head, item)

            if item in self.index:
                raise Exception("Cannot insert duplicate entries into the index")

            self.index[item] = tail
            
            if self.head is None:
                self.head = tail
                self.head.next = self.head

            if old_tail is not None:
                old_tail.next = tail

    def __iter__(self) -> Iterator[T]:
        head = self.head
        current = head
        while True:
            yield current.value
            current = current.next
            if current.value == head.value:
                break
    
    def __getitem__(self, offset: int) -> T:
        current = self.head
        for i in range(0, offset):
            current = current.next

        return current.value

    def __len__(self) -> int:
        return len(self.index)

    def __contains__(self, item: T) -> bool:
        return item in self.index

    def current(self) -> T:
        return self.head.value

    def next(self):
        self.head = self.head.next

    def reset(self, item: T):
        self.head = self.index[item]

    def after(self, item: T) -> Iterator[T]:
        head = self.index[item]
        current = head.next
        while True:
            yield current.value
            current = current.next
            if current.value == head.value:
                break

    def take_n_after(self, item: T, count: int) -> Tuple[T]:
        item_node = self.index[item]
        output = []
        for i in range(count):
            current = item_node.next
            del self.index[current.value]
            output.append(current.value)
            item_node.next = current.next

        return tuple(output)

    def insert_n_after(self, item: T, items: Iterable[T]):
        item_node = self.index[item]
        for item in reversed(items):
            new_node = CircularBufferNode(item_node.next, item)
            self.index[item] = new_node
            item_node.next = new_node

test_buffer = CircularBuffer([1, 2, 3, 4, 5])
assert list(test_buffer) == [1, 2, 3, 4, 5]
assert list(test_buffer.after(3)) == [4, 5, 1, 2]
assert test_buffer.current() == 1
test_buffer.next()
assert test_buffer.current() == 2
test_buffer.reset(1)
assert test_buffer.current() == 1

assert list(test_buffer.take_n_after(3, 2)) == [4, 5]
assert 4 not in test_buffer
assert 5 not in test_buffer
assert list(test_buffer) == [1, 2, 3]
test_buffer.insert_n_after(3, [4, 5])
assert list(test_buffer) == [1, 2, 3, 4, 5]
assert 4 in test_buffer

In [28]:
class Game(object):
    def __init__(self, init: Iterable[int], debug: bool = False):
        self.debug = debug
        self.buffer = CircularBuffer(init)
        self.move = 0
        self.min = min(self.buffer)
        self.max = max(self.buffer)

    def do_steps(self, number: int):
        for i in range(number):
            self.step()

    def step(self):
        self.move += 1
        if self.debug:
            print(f"-- move {self.move} --")


        # The current cup is located at the current position
        current_cup = self.buffer.current()

        if self.debug:
            print(f"cups: ({current_cup}) {' '.join(f' {cup} ' for cup in self.buffer.after(current_cup))}")

        # Pick up three cups which are immediately clockwise of the current position
        cups = self.buffer.take_n_after(current_cup, 3)
        
        self.buffer.next()

        if self.debug:
            print(f"pick up: {', '.join(map(str, cups))}")

        for offset in range(1, self.max):
            dest_cup = (current_cup - offset) % (2 + self.max - self.min)

            if dest_cup not in self.buffer:
                continue

            if self.debug:
                print(f"destination: {dest_cup}")
                print("")

            self.buffer.insert_n_after(dest_cup, cups)
            return

    def final_state(self) -> str:
        return "".join(map(str, self.buffer.after(1)))

    def part2_answer(self):
        next_cups = self.buffer.after(1)
        return next(next_cups) * next(next_cups)

example_game = Game(map(int, example), debug=True)
example_game.do_steps(10)
print(f"Example Final State after 10 moves (Part 1): {example_game.final_state()}")
example_game.debug = False
example_game.do_steps(90)
print(f"Example Final State after 100 moves (Part 1): {example_game.final_state()}")

true_game = Game(map(int, data))
true_game.do_steps(100)
print(f"Final State after 100 moves (Part 1): {true_game.final_state()}")

-- move 1 --
cups: (3)  8   9   1   2   5   4   6   7 
pick up: 8, 9, 1
destination: 2

-- move 2 --
cups: (2)  8   9   1   5   4   6   7   3 
pick up: 8, 9, 1
destination: 7

-- move 3 --
cups: (5)  4   6   7   8   9   1   3   2 
pick up: 4, 6, 7
destination: 3

-- move 4 --
cups: (8)  9   1   3   4   6   7   2   5 
pick up: 9, 1, 3
destination: 7

-- move 5 --
cups: (4)  6   7   9   1   3   2   5   8 
pick up: 6, 7, 9
destination: 3

-- move 6 --
cups: (1)  3   6   7   9   2   5   8   4 
pick up: 3, 6, 7
destination: 9

-- move 7 --
cups: (9)  3   6   7   2   5   8   4   1 
pick up: 3, 6, 7
destination: 8

-- move 8 --
cups: (2)  5   8   3   6   7   4   1   9 
pick up: 5, 8, 3
destination: 1

-- move 9 --
cups: (6)  7   4   1   5   8   3   9   2 
pick up: 7, 4, 1
destination: 5

-- move 10 --
cups: (5)  7   4   1   8   3   9   2   6 
pick up: 7, 4, 1
destination: 3

Example Final State after 10 moves (Part 1): 92658374
Example Final State after 100 moves (Part 1): 67384529
Final Stat

In [29]:
seed_cups = list(map(int, data))
max_seed = max(seed_cups)

cups = [
    *seed_cups,
    *range(max_seed + 1, int(1e6) + 1)
]

final_game = Game(cups)
assert len(final_game.buffer) == 1e6

final_game.do_steps(int(10e6))
print(f"Product of Next 2 Cups after 10M moves (Part 2): {final_game.part2_answer()}")

Product of Next 2 Cups after 10M moves (Part 2): 565615814504
